In [7]:
%env CUDA_VISIBLE_DEVICES=4,5,6,7
%env N_GPU=4

env: CUDA_VISIBLE_DEVICES=4,5,6,7
env: N_GPU=4


In [8]:
import os
import json
import csv
import torch
from tqdm import tqdm
import random
import copy
import wandb

from api_config import CONFIG

try: import google.colab; root_path = 'dsp'
except: root_path = '.'
import dsp

## wandb setup
os.environ["WANDB_API_KEY"]=CONFIG['wandb_api_key']
os.environ["WANDB_WATCH"]='false'
os.environ["WANDB_START_METHOD"]='thread'
os.environ["WANDB_USER_EMAIL"]=CONFIG['wandb_user_email']
os.environ["WANDB_USERNAME"]=CONFIG['wandb_username']

/home/chanwoong/.conda/envs/handbook/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def get_document_list(ins, ctxs_key='ctxs', n_docs=100):
    docs = ins[ctxs_key]

    document_list = []
    for i in range(n_docs):
        if ctxs_key == 'context':
            title = docs[i][0]
            text = docs[i][1]
        else:
            title = docs[i]['title']
            text = docs[i]['text']    
        
        document_list.append(docs[i])

    return document_list

def chunk_document_list(document_list, segment_size=5):
    assert len(document_list) % segment_size == 0

    chunked_list = []
    for i in range(0, len(document_list), segment_size):
        chunked_list.append(document_list[i:i + segment_size])
    
    return chunked_list


def create_prompt(example, iteration, document_input, prev_input):

    instruction="Follow instructions below\n1. Generate a summary of reference sentence and chosen sentences (prefix: Summary). Ensure the summary is under 200 words and does not include any pronouns. DO NOT make assumptions or attempt to answer the question; your job is to summarize only.\n2. Evaluate the summary based solely on the information of it, without any additional background context: if it lacks sufficient details to answer the question, print '[INCOMPLETE]'. If it provides all necessary details, print '[COMPLETE]'. You should provide the reason of evalution (prefix: Evaluation)\nSummary:"
    
    prompt = f"Source documents: {document_input}\n\nReference sentence: {prev_input}\n\nQuestion: {example['question']}\n\n{instruction}"
    
    return prompt

In [12]:
retrieval_results = []
with open('/ssd0/chanwoong/alignment-handbook/comp/data/retrieval/train_10000.json', 'r') as f:
    for line in f.readlines():
        retrieval_results.append(json.loads(line))

results = json.load(open('/ssd0/chanwoong/alignment-handbook/comp/data/generate/results_7500.json'))

In [ ]:
n_docs = 30
segment_size = 5

for result, retrieval in tqdm(zip(results, retrieval_results)):
    question = result['question']
    answer = result['answer']

    document_list = get_document_list(retrieval, n_docs=n_docs)
    chunk_list = chunk_document_list(document_list, segment_size=5)
    
    assert len(chunk_list) == len(result['iterations'])

    for idx, (iteration, chunk) in enumerate(zip(result['iterations'], chunk_list)):
        document_input = "\n".join(chunk_list)

        if idx == 0:
            prev_input = ""
        else:
            prev_input = result


        prompt = create_prompt(result, iteration, document_input, prev_input)
        
        